In [ ]:
from google.cloud.storage import Client
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
from pathlib import Path

from glambie.util.timeseries_helpers import derivative_to_cumulative

import seaborn as sns
sns.set_theme(palette='muted', style='whitegrid', font_scale=1.2)

In [ ]:
DATA_TRANSFER_BUCKET_NAME = "glambie-submissions"
PROJECT_NAME = "glambie"
storage_client = Client(project=PROJECT_NAME)

In [ ]:
list_of_blobs_in_bucket = storage_client.list_blobs(DATA_TRANSFER_BUCKET_NAME)
downloaded_files = []

In [ ]:
for blob in list_of_blobs_in_bucket:
    if 'glambie_csvs_to_publish_v2.zip' in blob.name:
        downloaded_files.append(blob.name)
        destination_file_path = os.path.join(local_path, blob.name)
        Path(destination_file_path).parent.mkdir(parents=True, exist_ok=True)
        with open(destination_file_path, "wb") as output_file:
            blob.download_to_file(output_file, raw_download=False)

In [ ]:
temporal_option = 'hydrological_years'
data_directory = local_path + 'glambie_csvs_to_publish' + '/' + temporal_option

In [ ]:
list_of_csvs = sorted(glob.glob(os.path.join(data_directory, '*.csv')))

In [ ]:
# choose a region
region = 'arctic_canada_south'

In [ ]:
for filename in list_of_csvs:
    if region in filename:
        region_dataframe = pd.read_csv(filename)